In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
sys.path.insert(0, os.getenv('SRC_PATH'))

import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
from src.volsurface import GridInterpVolSurface, KernelVolSurface
from src.utils.data_helper import clean_data, VolSurfPointwiseDataset

import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from src.train import Trainer

import json

DB_PATH = os.getenv('DB_PATH')
CSV_PATH = os.getenv('CSV_PATH')
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

pd.set_option('future.no_silent_downcasting', True)

/Users/zhanghq/miniconda3/envs/vae-volsurface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-18 12:05:48,801	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-18 12:05:48,895	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from src.utils.logger import setup_logger
logger = setup_logger('train')

In [3]:
query = """
SELECT date, symbol, exdate, last_date, cp_flag, strike_price, best_bid, best_offer, volume, open_interest, impl_volatility, delta
FROM opprc
"""
dtype = {
    'symbol': 'string',
    'cp_flag': 'string',
    'strike_price': 'float64',
    'best_bid': 'float64',
    'best_offer': 'float64',
    'volume': 'int64',
    'open_interest': 'int64',
    'impl_volatility': 'float64',
    'delta': 'float64'
}
df_raw = pd.read_sql_query(query, conn, parse_dates=['date', 'exdate', 'last_date'])
df_raw = df_raw.replace('', np.nan) # sqlite returns empty strings for NULL values
df_raw = df_raw.astype(dtype)

df = clean_data(df_raw)

[2025-04-18 11:56:13] [INFO] src.utils.data_helper (49) : Bad data - Filtered 1110238 rows, Retained sample 76.67%
[2025-04-18 11:56:25] [INFO] src.utils.data_helper (62) : Consecutive trading stats completed
[2025-04-18 11:56:27] [INFO] src.utils.data_helper (85) : Consecutive trading - Filtered 2341950 rows, Retained sample 35.81%
[2025-04-18 11:56:27] [INFO] src.utils.data_helper (90) : Moneyness calculation completed


In [3]:
model_name = "vae_pw_improve"
train_model = True
load_model = False
save_model = False
data_dir = CSV_PATH + "/predicted_vol_surfaces.json"  # Path to the volatility surfaces dataset
batch_size = 32
epochs = 4

In [4]:
maturity_grid = np.array([1, 7, 30, 60, 90, 180, 360, 720])
delta_grid = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [5]:
SRC_PATH = os.getenv('SRC_PATH')
os.chdir(SRC_PATH)
trainer = Trainer(model_name)
trainer.create_model()

[2025-04-18 12:05:55] [INFO] src.train (62) : Using device: mps


In [7]:
with open(data_dir, "r") as f:
    data = json.load(f)

In [8]:
mapping_ids = {dt: i for i, dt in enumerate(sorted(map(lambda x: x[:10], data.keys())))}

In [9]:
df['mapping_ids'] = df['date'].dt.strftime('%Y-%m-%d').map(mapping_ids)

In [10]:
vol_surfaces = []
for key in sorted(data.keys()):
    surface = torch.tensor(data[key], dtype=torch.float32)
    vol_surfaces.append(surface.flatten())  # Flatten 2D to 1D

data_tensor = torch.stack(vol_surfaces)
pw_grid_data = torch.tensor(df[['ttm', 'moneyness']].values, dtype=torch.float32)
# !only for test run
pw_grid_data[:, 0] = pw_grid_data[:, 0] / 365.0
pw_vol_data = torch.tensor(df['impl_volatility'].values, dtype=torch.float32)
mapping_ids = torch.tensor(df['mapping_ids'].values).long()

assert data_tensor.shape[0] == max(mapping_ids) + 1

In [11]:
dataset = VolSurfPointwiseDataset(pw_grid_data, pw_vol_data, data_tensor, mapping_ids)
train_loader = DataLoader(
    dataset, 
    batch_size=trainer.batch_size,
    shuffle=True
)

In [13]:
# Train the model
for epoch in range(epochs):
    logger.info(f"Epoch {epoch + 1}/{epochs}")
    trainer.train(train_loader)

[2025-04-18 11:58:16] [INFO] train (3) : Epoch 1/4
[2025-04-18 11:59:33] [INFO] src.train (181) : Loss: 0.0063
[2025-04-18 11:59:33] [INFO] train (3) : Epoch 2/4
[2025-04-18 12:00:49] [INFO] src.train (181) : Loss: 0.0043
[2025-04-18 12:00:49] [INFO] train (3) : Epoch 3/4
[2025-04-18 12:02:06] [INFO] src.train (181) : Loss: 0.0040
[2025-04-18 12:02:06] [INFO] train (3) : Epoch 4/4
[2025-04-18 12:03:23] [INFO] src.train (181) : Loss: 0.0039


In [14]:
torch.save(trainer.model.state_dict(), f"params/{trainer.model_name}.pth")

## Test generation

In [6]:
from src.volsurface import VAEPWVolSurface

In [7]:
trainer.load_model("params/vae_pw_improve.pth")

[2025-04-18 12:06:04] [INFO] src.train (138) : Model loaded from params/vae_pw_improve.pth


In [8]:
vaevsurf = VAEPWVolSurface(trainer.model)

In [24]:
vaevsurf.refresh()
vaevsurf.predict_grid(delta_grid, maturity_grid)

array([[ 0.3757671 ,  0.37278825,  0.17734627, -0.03573316, -0.74058443,
         0.20380884, -0.632482  ,  0.19291478],
       [ 0.4376036 ,  0.54978806,  0.17197211,  0.16721374, -0.54719895,
         0.21468495, -0.43909687,  0.39765185],
       [ 0.4611993 ,  0.61716294,  0.16965374,  0.2561105 , -0.4624254 ,
         0.21984306, -0.35432333,  0.48632354],
       [ 0.46819577,  0.6333496 ,  0.16894221,  0.28483468, -0.4352209 ,
         0.22131109, -0.32711846,  0.5147168 ],
       [ 0.4732299 ,  0.64034903,  0.16863441,  0.297818  , -0.4229092 ,
         0.22193179, -0.31480724,  0.5265611 ],
       [ 0.4799876 ,  0.64756715,  0.16937657,  0.31154352, -0.41003722,
         0.22269131, -0.30193502,  0.5384483 ],
       [ 0.48800758,  0.65681016,  0.17036815,  0.32939273, -0.39424747,
         0.22368291, -0.28614515,  0.5513165 ],
       [ 0.49632394,  0.66735494,  0.17148271,  0.3490048 , -0.37671536,
         0.22479746, -0.26861352,  0.56618476],
       [ 0.50426847,  0.67837024